note: `os.environ["AD_PATH"]` shoule be set before importing `ad.module`

In [1]:
from ad.scalar import *
from ad.module import *
from mlir.ir import Context, Location

define function that has `Scalar`(s) as input(s) and `Scalar`(s) as output(s)

In [2]:
def function(x: Scalar, y: Scalar) -> Scalar:
    return x.log() + x * y - y

call `pyfunc_to_mlir` to transform python function to mlir module

In [3]:
with Context() as ctx, Location.unknown() as loc:
    primal_ir = pyfunc_to_mlir(ctx, loc, 2, 1, "function", function)
    
print(primal_ir)

module {
  func.func @function(%arg0: tensor<1xf32>, %arg1: tensor<1xf32>) -> tensor<1xf32> {
    %0 = "tosa.log"(%arg0) : (tensor<1xf32>) -> tensor<1xf32>
    %1 = "tosa.mul"(%arg0, %arg1) {shift = 0 : i32} : (tensor<1xf32>, tensor<1xf32>) -> tensor<1xf32>
    %2 = "tosa.add"(%0, %1) : (tensor<1xf32>, tensor<1xf32>) -> tensor<1xf32>
    %3 = "tosa.sub"(%2, %arg1) : (tensor<1xf32>, tensor<1xf32>) -> tensor<1xf32>
    return %3 : tensor<1xf32>
  }
}



initialize `ScalarModule` with generated mlir

In [4]:
primal_module = ScalarModule(primal_ir, "function")
print(primal_module)

module {
  func.func @function(%arg0: tensor<1xf32>, %arg1: tensor<1xf32>) -> tensor<1xf32> {
    %0 = "tosa.log"(%arg0) : (tensor<1xf32>) -> tensor<1xf32>
    %1 = "tosa.mul"(%arg0, %arg1) {shift = 0 : i32} : (tensor<1xf32>, tensor<1xf32>) -> tensor<1xf32>
    %2 = "tosa.add"(%0, %1) : (tensor<1xf32>, tensor<1xf32>) -> tensor<1xf32>
    %3 = "tosa.sub"(%2, %arg1) : (tensor<1xf32>, tensor<1xf32>) -> tensor<1xf32>
    return %3 : tensor<1xf32>
  }
}



call `ScalarModule.run` to exec

In [5]:
x = 2.0
y = 5.0
grad = 1.0

primal_res = primal_module.run(x, y)
print(primal_res)

5.6931477


call `ScalarModule.grad` to generate adjoint module

In [6]:
adjoint_module = primal_module.grad()
print(adjoint_module)

module {
  func.func @diff_function(%arg0: tensor<1xf32>, %arg1: tensor<1xf32>, %arg2: tensor<1xf32>) -> (tensor<1xf32>, tensor<1xf32>, tensor<1xf32>) {
    %cst = arith.constant dense<1.000000e+00> : tensor<1xf32>
    %0 = "tosa.log"(%arg0) {requires_grad = 2 : i64} : (tensor<1xf32>) -> tensor<1xf32>
    %1 = "tosa.mul"(%arg0, %arg1) {requires_grad = 5 : i64, shift = 0 : i32} : (tensor<1xf32>, tensor<1xf32>) -> tensor<1xf32>
    %2 = "tosa.add"(%0, %1) {requires_grad = 3 : i64} : (tensor<1xf32>, tensor<1xf32>) -> tensor<1xf32>
    %3 = "tosa.sub"(%2, %arg1) {requires_grad = 4 : i64} : (tensor<1xf32>, tensor<1xf32>) -> tensor<1xf32>
    %4 = "tosa.negate"(%cst) : (tensor<1xf32>) -> tensor<1xf32>
    %5 = "tosa.mul"(%4, %arg2) {shift = 0 : i32} : (tensor<1xf32>, tensor<1xf32>) -> tensor<1xf32>
    %6 = "tosa.reciprocal"(%arg0) : (tensor<1xf32>) -> tensor<1xf32>
    %7 = "tosa.mul"(%6, %arg2) {shift = 0 : i32} : (tensor<1xf32>, tensor<1xf32>) -> tensor<1xf32>
    %8 = "tosa.mul"(%arg1, %

call `adjoint_module.exec` to exec (inputs should be forward inputs and reverse contribution)

In [7]:
adjoint_res = adjoint_module.run(x, y, grad)
print(adjoint_res)

[5.6931477, 5.5, 1.0]


check if answers are correct

In [8]:
from math import log
from typing import Tuple

epsilon = 1e-6

def forward(x: float, y: float) -> float:
    return log(x) + x * y - y

def reverse(x: float, y: float, grad: float) -> Tuple[float]:
    forward_res = forward(x, y)
    x_grad = 1 / x + y
    y_grad = x - 1
    return [forward_res, x_grad, y_grad]

handwritten_res = reverse(x, y, grad)
for i in range(len(handwritten_res)):
    assert abs(handwritten_res[i] - adjoint_res[i]) < epsilon, "Wrong answer"